In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [8]:
#https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source, 'lxml')  
table=soup.find('table')
table_rows=table.findAll('tr')
#l is the list to be passed through the function to obtain dataframe
l = []
for tr in table_rows:
    td = tr.find_all('td')
    raw_row = [tr.text for tr in td]
    #to check for empty rows and exclude these cases
    if raw_row == [] or raw_row[1] == 'Not assigned':
        continue

    #to parse neighborhood rows to remove \n
    edited_row=raw_row[0:len(raw_row)-1]
    edited_row.append(raw_row[-1][:-1])
    
    #check for empty neighborhoods to be replaced with borough 
    if edited_row[2] == 'Not assigned':
        edited_row[2] = edited_row[1]
    
    #adding entry to l
    l.append(edited_row)
    
    
df=pd.DataFrame(l, columns=["Postcode", "Borough", 'Neighborhood'])

df2=df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
display(df2)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df2.shape


(103, 3)

Get CSV files from http://cocl.us/Geospatial_data

In [14]:
geo_data=pd.read_csv("http://cocl.us/Geospatial_data")

geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
full_table = df2.set_index('Postcode').join(f_geo.set_index('Postal Code'))
full_table = full_table.sample(frac=1).reset_index(drop=True)
full_table.head(20)

,Borough,Neighborhood,Latitude,Longitude
0,North York,Willowdale West,43.782736,-79.442259
1,North York,Victoria Village,43.725882,-79.315572
2,York,Weston,43.706876,-79.518188
3,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
4,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
5,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
6,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
7,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,North York,"Northwood Park, York University",43.767980,-79.487262
9,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
